#### Notebook to download trade ticks from Interactive Brokers API

In [ ]:
# Read in list of tickers
import pandas as pd
at = pd.read_csv('..\\..\\data\\available_tickers.csv')
at['0']

In [ ]:
# MUST HAVE IB DESKTOP APP OPEN AND LOGGED IN TO ACCESS API
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import ListOfHistoricalTickLast
import threading
import time
import datetime as dt
import os

class IBapi(EWrapper, EClient):
    def __init__(self, last_time):
        self.last_time = last_time
        self.current_symbol = None
        EClient.__init__(self, self)
        
    def historicalTicksLast(self, reqId: int, ticks: ListOfHistoricalTickLast, done: bool):
        if done:

            if len(ticks) > 0:
                # here will return 1000+ ticks to be processed, or 0 ticks with no error -> increment time
                print('lenticks',len(ticks))
                data = [
                    (t.time,
                     t.time,
                     t.tickAttribLast if t.tickAttribLast.pastLimit or t.tickAttribLast.unreported else None,
                     t.price, 
                     t.size, 
                     t.exchange, 
                     t.specialConditions)
                    for t in ticks
                ]
                ticks_df = pd.DataFrame(
                    data,
                    columns=['Time', 'TimeStamp', 'TickAttribLast', 'Price', 'Size', 'Exchange', 'SpecialConditions'],
                )
                ticks_df['Time']=pd.to_datetime(ticks_df['Time'], unit='s', utc=True)
                ticks_df['Time'] = ticks_df['Time'].dt.tz_convert('Europe/Oslo')

                end_time = ticks_df['Time'].max()
                
                filename = self.current_symbol + '_' + end_time.strftime('%Y-%m') + '_TRADES.csv'
                ticks_df.to_csv('..\\..\\data\\TICKS\\' + filename, mode='a', header=not os.path.exists(filename), index=False)
            else:
                end_time = dt.datetime.fromtimestamp(self.last_time)

            if len(ticks) < 1000:
                # hop to next day
                lt = dt.datetime.combine(end_time, dt.datetime.min.time())
                self.last_time = dt.datetime.timestamp(lt + dt.timedelta(days=1))
            else:

                self.last_time = dt.datetime.timestamp(end_time) + 1

# ONLY APPROX. 3 MONTHS OF TICK DATA IS AVAILABLE AT A TIME
initial_time = dt.datetime(2022, 1, 1, 0, 0, 0)
app = IBapi(last_time = dt.datetime.timestamp(initial_time))
app.connect('127.0.0.1', 7496, 129)


def run_loop():
    app.run()

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

# LOOP THROUGH EACH TICKER
for ticker in at['0']:
    print(ticker)
    #Create contract object
    stock = Contract()
    stock.symbol = ticker
    stock.secType = 'STK'
    stock.exchange = 'OSE'
    stock.currency = 'NOK'
    
    start_time = initial_time
    while start_time < dt.datetime(2022, 3, 31, 23, 59, 59):# dt.datetime.now():

        app.current_symbol = stock.symbol

        print('starttime',start_time.strftime('%Y%m%d %H:%M:%S'))
        app.reqHistoricalTicks(2, stock, start_time.strftime('%Y%m%d %H:%M:%S'), "", 1000, "TRADES", 0, True, [])
        time.sleep(2.7) #sleep to allow enough time for data to be returned

        start_time = dt.datetime.fromtimestamp(app.last_time)

app.disconnect()
